In [ ]:
# Import dependencies
import pandas as pd
import random as random
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import linregress
from scipy.stats import f_oneway
import requests
import json 


import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
#Jeff Code

#Functions to be used for code

#Function #1
#plotting routine with regression (setting up as a function)
#below function takes 2 sets of x and y data and generates a scatter plot for each.  Then performs a linear regression 
#and returns a dataframe with the regression results
#x and y are number pairs
#data is the name of what is being plotted (like temperature).  used as Y axis label and as part of title
#prints results of regression below graphs

def plotter(x1,y1,Data1,x2,y2,Data2):
   
   #plotting data set x1 and y1

    (slope_N, intercept_N, rvalue_N, pvalue_N, stderr_N) = linregress(x1,y1)
    regress_values_N = x1 * slope_N + intercept_N
    line_eq = "y = " + str(round(slope_N,3)) + "x + " + str(round(intercept_N,3))+", r = " +str(round(rvalue_N,3))\
                        + " p = "+str(round(pvalue_N,3))
    max_y = y1.max()
    min_x = x1.min()
    fig = plt.figure(figsize = (15,4))
    ax1 = fig.add_subplot(121)
    ax1.scatter(x1, y1, s=5)
    plt.plot(x1,regress_values_N,"r-")
    plt.annotate(line_eq,(min_x+1,max_y-1),fontsize=11,color="red") #writes to coordinates on graph
    plt.xlabel("Year")
    plt.ylabel(f"{Data1} ")
    plt.title(f"{Data1} vs Year")

  #plotting data set x2 and y2
    (slope_S, intercept_S, rvalue_S, pvalue_S, stderr_S) = linregress(x2,y2)
    regress_values_S = x2 * slope_S + intercept_S
    line_eq = "y = " + str(round(slope_S,3)) + "x + " + str(round(intercept_S,3))+", r = " +str(round(rvalue_S,3))\
                        + " p = "+str(round(pvalue_S,3))
    max_y = y2.max()
    min_x = x2.min()
    #plotting second data set
    ax2 = fig.add_subplot(122)
    ax2.scatter(x2, y2, s=5)
    plt.plot(x2,regress_values_S,"r-")
    plt.annotate(line_eq,(min_x+1,max_y-1),fontsize=11,color="red") #writes to coordinates on graph
    plt.xlabel("Year")
    plt.ylabel(f"{Data2}")
    plt.title(f"{Data2} vs Year")
    
    plt.show()


    Result_Temp_df = pd.DataFrame({"Slope":[slope_N, slope_S], "Intercept":[intercept_N, intercept_S], 
                                                         "r-value":[rvalue_N, rvalue_S], "p-value":[pvalue_N, pvalue_S]})

    Result_Temp_df["Param"] =[Data1, Data2]
    Result_Temp_df = Result_Temp_df[["Param", "Slope", "Intercept", "r-value", "p-value"]]

    print(f"Correlation Results")
    print("")

    print(Result_Temp_df)
    return Result_Temp_df;

In [ ]:
# Kristina's code starts here

In [ ]:
# Kristina's code ends here

In [ ]:
# Jeff's code starts here

#reading in raw data
temp_data_df = pd.read_csv('Resources/clean_temp_data.csv')
#temp_data_df.head(3)


In [ ]:
#to understand impact of removing data on column length
print("Results of Cleaning up Data (removing NaN)")
print("")
names = temp_data_df.columns
print("Before")
for i in names:
    full = temp_data_df[i].count()
    Null = temp_data_df[i].isnull().sum(axis = 0)
    print(f"for {i} column: Full {full} vs Empty or NaN {Null}")
    


In [ ]:
#Final clean up of Data

#removing NaN
Start_Length = len(temp_data_df["Date"])
temp_data_df.dropna(inplace = True)

#temp_data_df
Finish_Length = len(temp_data_df["Date"])
print(f"Temperature and precipitation data starts with {Start_Length} records")
print(f"After removing rows with empty data fields have {Finish_Length} records")

#Creating Seasonal Averages
    #Winter - Dec, Jan, Feb
    #Spring - March, April, May
    #Summer - June, July, August
    #Fall - Sept, Oct, Nov

#code below combines all locations into one average value for month and year
list = temp_data_df["Name"].unique()
temp_comb_df = temp_data_df.groupby(["Date"])["Total Monthly Precipitation", "Total Montly Snowfall (mm)",
                                             "Avg Monthly Temp", "Monthly Mean Max Temp",
                                              "Monthly Mean Min Temp"].mean().reset_index()

#Data is string Year-Month (month two digits 01-12).  This splits into two fields
temp_comb_df["Month"] = temp_comb_df["Date"].str.split("-").str[1].astype(int)
temp_comb_df["Year"] = temp_comb_df["Date"].str.split("-").str[0].astype(int)

col_list = temp_comb_df.columns.tolist()
Season_data_df = pd.DataFrame(columns = col_list)

#combining months to form seasons: Winter (Dec-Feb), Fall(Mar-May), Summer(June-Aug), Fall(Sept-Nov)
for i in temp_comb_df["Year"].unique():
    if i!= 1950 and i!= 2020: #ignore 1950 because don't have dec 1949 and don't have all 2020
        
        #in dataframe the "month" represents the season: 2=winter, 4 = spring, 7 = summer, 10=fall
        #winter data is complicated because Dec is previous year
        winter_data = temp_comb_df.loc[((temp_comb_df["Year"]==i) & (temp_comb_df["Month"].between(1,2)))|\
                                         ((temp_comb_df["Year"]==(i-1)) & (temp_comb_df["Month"]==12))].mean()
        winter_data["Year"]=winter_data["Year"].astype(int)+1 # extra 1 because rounds down to year previous
        winter_data["Month"] = 1 #would be 5 otherwise (mean(12,1,2))
        
        #isolating spring data
        spring_data = temp_comb_df.loc[((temp_comb_df["Year"]==i) & (temp_comb_df["Month"].between(3,5)))].mean()
        spring_data["Year"]=spring_data["Year"].astype(int)
        
        #Isolating summer data
        summer_data = temp_comb_df.loc[((temp_comb_df["Year"]==i) & (temp_comb_df["Month"].between(6,8)))].mean()
        summer_data["Year"]=summer_data["Year"].astype(int)
        
        #isolating fall
        fall_data = temp_comb_df.loc[((temp_comb_df["Year"]==i) & (temp_comb_df["Month"].between(9,11)))].mean() 
        fall_data["Year"]=fall_data["Year"].astype(int)
        
        #combining together all seasons into one data frame
        frames = [winter_data, spring_data, summer_data, fall_data]
        Season_data_df = Season_data_df.append(frames, ignore_index = True)
        
winter = 1
sprint = 4
summer = 7
fall = 10
Season_data_df["Temp Swing"] = Season_data_df["Monthly Mean Max Temp"]-Season_data_df["Monthly Mean Min Temp"]
Season_data_df.drop(columns=["Date"], inplace = True)


In [ ]:
Data1 = "Avg Monthly Temp"
x1 =  Season_data_df[Season_data_df["Month"]==summer]["Year"]
y1 =  Season_data_df[Season_data_df["Month"]==summer][Data1]
Data2 = "Avg Monthly Temp"
x2 =  Season_data_df[Season_data_df["Month"]==winter]["Year"]
y2 =  Season_data_df[Season_data_df["Month"]==winter][Data2]
print("")
print(f"Avg Temperatures (F) for Winter and Summer Months")
print("")
Data1 = Data1 + " (Summer)"
Data2= Data2 + " (Winter)"
Correlation_df = plotter(x1,y1,Data1,x2,y2,Data2)


In [ ]:
Data1 = "Monthly Mean Min Temp"
x1 =  Season_data_df[Season_data_df["Month"]==winter]["Year"]
y1 =  Season_data_df[Season_data_df["Month"]==winter][Data1]
Data2 = "Monthly Mean Max Temp"
x2 =  Season_data_df[Season_data_df["Month"]==winter]["Year"]
y2 =  Season_data_df[Season_data_df["Month"]==winter][Data2]
print("")
print(f"Minimum and Maximum Avg Temperatures for Winter Months")
print("")
Data1 = Data1 + " (Winter)"
Data2= Data2 + " (Winter)"
Correlation_df = plotter(x1,y1,Data1,x2,y2,Data2)

In [ ]:
Data1 = "Temp Swing"
x1 =  Season_data_df[Season_data_df["Month"]==winter]["Year"]
y1 =  Season_data_df[Season_data_df["Month"]==winter][Data1]
Data2 = "Temp Swing"
x2 =  Season_data_df[Season_data_df["Month"]==summer]["Year"]
y2 =  Season_data_df[Season_data_df["Month"]==summer][Data2]
print("")
print(f"Avg Temperature Swings for Winter and Summer (Seasonal max - seasonal min)")
print("")
Data1 = Data1 +" Winter"
Data2 = Data2 +" Summer"
Correlation_df = plotter(x1,y1,Data1,x2,y2,Data2)

In [ ]:
#Analysis for Winter
#ANOVA
#binning data for analysis
year_min = Season_data_df["Year"].min()
year_max = Season_data_df["Year"].max()
delta_year = year_max-year_min
delta_year
bin_size = 17 #number of years

bins = [(year_min+i*bin_size) for i in range(0,int(delta_year/bin_size)+1)]
groups = [((i-1)*bin_size+(1951+bin_size/2)) for i in range(1,int(delta_year/bin_size)+1)]
Season_data_df["Bin"]=pd.cut(Season_data_df["Year"],bins, labels = groups, include_lowest = True)

#plotting average for bins
x_label = groups
y_label = [Season_data_df[(Season_data_df["Bin"]==i)&(Season_data_df["Month"]==winter)]["Temp Swing"].mean() for i in groups]
plt.scatter(x_label,y_label,s = 20)
plt.title(f"Temp Swing (Max-Min) for Winter (bin size {bin_size} years)")
plt.xlabel(f"Year bin - size {bin_size} years")
plt.ylabel('Temp Swing (Max-Min)')
plt.show()


#box plot of full data in bins
red_diamond = dict(markerfacecolor='r', marker='D')
fig1, ax1 = plt.subplots()
ax1.set_title('Temperature Swing in Winter')
ax1.set_ylabel('Temp Swing (Ma)')
ax1.set_xlabel(f"Year bin - size {bin_size} years")

ax1.boxplot([Season_data_df[(Season_data_df["Bin"]==i)&(Season_data_df["Month"]==winter)]["Temp Swing"] for i in groups], labels=groups,showfliers = True,
            flierprops = red_diamond)
plt.show()

bin_1 = Season_data_df[Season_data_df["Bin"]==groups[0]]["Temp Swing"].tolist()
bin_2 = Season_data_df[Season_data_df["Bin"]==groups[1]]["Temp Swing"].tolist()
bin_3 = Season_data_df[Season_data_df["Bin"]==groups[2]]["Temp Swing"].tolist()
bin_4 = Season_data_df[Season_data_df["Bin"]==groups[3]]["Temp Swing"].tolist()
f_oneway(bin_1, bin_2, bin_3, bin_4)

In [ ]:
#Analysis for Summer
#ANOVA
#binning data for analysis
year_min = Season_data_df["Year"].min()
year_max = Season_data_df["Year"].max()
delta_year = year_max-year_min
delta_year
bin_size = 17 #number of years

bins = [(year_min+i*bin_size) for i in range(0,int(delta_year/bin_size)+1)]
groups = [((i-1)*bin_size+(1951+bin_size/2)) for i in range(1,int(delta_year/bin_size)+1)]
Season_data_df["Bin"]=pd.cut(Season_data_df["Year"],bins, labels = groups, include_lowest = True)

#plotting average for bins
x_label = groups
y_label = [Season_data_df[(Season_data_df["Bin"]==i)&(Season_data_df["Month"]==summer)]["Temp Swing"].mean() for i in groups]
plt.scatter(x_label,y_label,s = 20)
plt.title(f"Temp Swing (Max-Min) for Winter (bin size {bin_size} years)")
plt.xlabel(f"Year bin - size {bin_size} years")
plt.ylabel('Temp Swing (Max-Min)')
plt.show()


#box plot of full data in bins
red_diamond = dict(markerfacecolor='r', marker='D')
fig1, ax1 = plt.subplots()
ax1.set_title('Temperature Swing in Winter')
ax1.set_ylabel('Temp Swing (Ma)')
ax1.set_xlabel(f"Year bin - size {bin_size} years")

ax1.boxplot([Season_data_df[(Season_data_df["Bin"]==i)&(Season_data_df["Month"]==summer)]["Temp Swing"] for i in groups], labels=groups,showfliers = True,
            flierprops = red_diamond)
plt.show()

bin_1 = Season_data_df[Season_data_df["Bin"]==groups[0]]["Temp Swing"].tolist()
bin_2 = Season_data_df[Season_data_df["Bin"]==groups[1]]["Temp Swing"].tolist()
bin_3 = Season_data_df[Season_data_df["Bin"]==groups[2]]["Temp Swing"].tolist()
bin_4 = Season_data_df[Season_data_df["Bin"]==groups[3]]["Temp Swing"].tolist()
f_oneway(bin_1, bin_2, bin_3, bin_4)


In [ ]:
## Analyzing Precipitation

In [ ]:
#Analyzing Total Precipitation Each Year 
print(f"Regression Analysis for Total Yearly Precipitation versus Year")

precip_comb_df = temp_comb_df.groupby(["Year"])["Total Monthly Precipitation","Month"].sum().reset_index()
precip_comb_df = precip_comb_df.rename(columns = {"Total Monthly Precipitation":"Total Yearly Precipitation (in)"})
precip_comb_df=precip_comb_df.drop(70) #dropping 2020 since incomplete


x_axis = precip_comb_df["Year"]
y_axis = precip_comb_df["Total Yearly Precipitation (in)"]

max_y = y_axis.max()
min_x = x_axis.min()
plt.scatter(x_axis,y_axis, s = 4)
(slope_N, intercept_N, rvalue_N, pvalue_N, stderr_N) = linregress(x_axis,y_axis)
regress_values_N = x_axis * slope_N + intercept_N
line_eq = "y = " + str(round(slope_N,3)) + "x + " + str(round(intercept_N,3))+", r = " +str(round(rvalue_N,3))\
                        + " p = "+str(round(pvalue_N,3))
plt.plot(x_axis,regress_values_N,"r-")
plt.annotate(line_eq,(min_x+1,max_y-1),fontsize=11,color="red") #writes to coordinates on graph
plt.title("Total Precipitation by Year")
plt.xlabel("Year")
plt.ylabel("Total Precipitation (inches)")  #need to double check units - temperature is F, would precip be inches

plt.show()

In [ ]:
type(x_axis[1])

In [ ]:
year_min = precip_comb_df["Year"].min()
year_max = precip_comb_df["Year"].max()
delta_year = year_max-year_min
delta_year
bin_size = 17 #number of years

bins = [(year_min+i*bin_size) for i in range(0,int(delta_year/bin_size)+1)]
groups = [((i-1)*bin_size+(1951+bin_size/2)) for i in range(1,int(delta_year/bin_size)+1)]
precip_comb_df["Bin"]=pd.cut(precip_comb_df["Year"],bins, labels = groups, include_lowest = True)

print(f"Analysis of Hypothesis for Precipitation")
print("")
#box plot of full data in bins
red_diamond = dict(markerfacecolor='r', marker='D')
fig1, ax1 = plt.subplots()
ax1.set_title('Precipitation over years')
ax1.set_ylabel('Precipitation (in)')
ax1.set_xlabel(f"Year bin - size {bin_size} years")

ax1.boxplot([precip_comb_df[(precip_comb_df["Bin"]==i)]["Total Yearly Precipitation (in)"] for i in groups], labels=groups,showfliers = True,
            flierprops = red_diamond)
plt.show()


print("")
bin_1 = precip_comb_df[precip_comb_df["Bin"]==groups[0]]["Total Yearly Precipitation (in)"].tolist()
bin_2 = precip_comb_df[precip_comb_df["Bin"]==groups[1]]["Total Yearly Precipitation (in)"].tolist()
bin_3 = precip_comb_df[precip_comb_df["Bin"]==groups[2]]["Total Yearly Precipitation (in)"].tolist()
bin_4 = precip_comb_df[precip_comb_df["Bin"]==groups[3]]["Total Yearly Precipitation (in)"].tolist()
print("f_oneway(bin_1, bin_2, bin_3, bin_4)")

print(f"The p-value is less than 5% so we reject the Null Hypothesis - Precipitation is Changing with Year")

In [ ]:
#Jeff's code ends here

In [ ]:
# Faduma's code starts here
df = pd.read_csv('Temperature_Data/Temp_Data.csv')

#Variables assigned to season
count = 0
winter = 1
spring = 4
summer = 7
fall = 10

#variables for data being found
summer_temp_avg = []
summer_temp_max = []
summer_temp_min = []
winter_temp_avg = []
winter_temp_max = []
winter_temp_min = []
spring_temp_avg = []
spring_temp_max = []
spring_temp_max = []
spring_temp_min = []
fall_temp_avg = []
fall_temp_min = []
fall_temp_max = []

for i in range(len(df["DATE"])):
    try:
        date = int(df["DATE"][i].split("-")[1])
#print(df["TMAX"][count]) & print(date)
        if date < 4:
            winter_temp_max.append(float(df["TMAX"][i]))
            winter_temp_min.append(float(df["TMIN"][i]))
            winter_temp_avg.append(float(df["TAVG"][i]))
        elif date < 7:
            spring_temp_max.append(float(df["TMAX"][i]))
            spring_temp_min.append(float(df["TMIN"][i]))
            spring_temp_avg.append(float(df["TAVG"][i]))

        elif date < 10:
            summer_temp_max.append(float(df["TMAX"][i]))
            summer_temp_min.append(float(df["TMIN"][i]))
            summer_temp_avg.append(float(df["TAVG"][i]))
        else:
            fall_temp_max.append(float(df["TMAX"][i]))
            fall_temp_min.append(float(df["TMIN"][i]))
            fall_temp_avg.append(float(df["TAVG"][i]))
        count += 1
    
    except:
        break
#printing (maybe into a neater chart?)
print("Max summer TMAX", max(summer_temp_max))
print("Min summer TMIN", min(summer_temp_min))
print("Avg summer TAVG", min(summer_temp_avg))
print("Max winter TMAX", max(winter_temp_max))
print("Min winter TMIN", min(winter_temp_min))
print("Avg winter TAVG", min(winter_temp_avg))

print("Max spring TMAX", max(spring_temp_max))
print("Min spring TMIN", min(spring_temp_min))
print("Avg spring TAVG", min(spring_temp_avg))
print("Max fall TMAX", max(fall_temp_max))
print("Min fall TMIN", min(fall_temp_min))
print("AVG fall TAVG", min(fall_temp_avg))




In [ ]:
#finding Avg Percepation VS year

temp_data_df = pd.read_csv('Temperature_Data/Temp_Data.csv')

#finding Avg Percepation VS year
Avg_pcp = []
year_dict = {}
for i in range(len(df["DATE"])):
    try:
        year = (df["DATE"][i]).split("-")[0]
        if year not in year_dict:
            year_dict[year] = []
            year_dict[year].append((df["PRCP"][i]))
        else:
            year_dict[year].append((df["PRCP"][i]))
    except:
        pass
    
#more variables:
year_lis = []
prcp_lis = []

for key in year_dict.keys():
    if key =="DATE":
        continue
    year_lis.append(key)
    sum_1 = 0.0
    #print(year_lis[key])
    for i in range(len(year_dict[key])):
        #print(year_dict[key][i])
        if(np.isnan(float(year_dict[key][i]))):
            continue
        try:
            sum_1 = sum_1 + float(year_dict[key][i])
        except:
            pass
    avg = sum_1/len(year_dict[key])
    prcp_lis.append(avg)

#print(year_lis)
#print(prcp_lis)

for i in range(0,len(year_lis)):
    year_lis[i] = float(year_lis[i]) 

year_lis = np.array(year_lis)#convert format so works incalculation for regression
min_x = min(year_lis)
max_y = max(prcp_lis)
get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt

plt.scatter(year_lis, prcp_lis,linestyle='-', marker='o') 
  
#x axis label
plt.xlabel('year') 
#y axis label
plt.ylabel('prcp avg (mm)') 
#title to my graph 
plt.title('year vs prcp') 

(slope_N, intercept_N, rvalue_N, pvalue_N, stderr_N) = linregress(year_lis,prcp_lis)
regress_values_N = year_lis * slope_N + intercept_N
line_eq = "y = " + str(round(slope_N,3)) + "x + " + str(round(intercept_N,3))+", r = " +str(round(rvalue_N,3))\
                        + " p = "+str(round(pvalue_N,3))
plt.plot(year_lis,regress_values_N,"r-")
plt.annotate(line_eq,(min_x+1,max_y-1),fontsize=11,color="red") #writes to coordinates on graph

#show the plot! 
plt.show()

# Faduma's code ends here

In [ ]:
df

In [ ]:
# Emily's code starts here

In [ ]:
storm_data = pd.read_csv('Resources/clean_storm_event_data.csv')
storm_data

In [ ]:
storm_data['Event Type'].unique()

# Counts of Tornado Eventsin Minnesota from 1950 to 2020

In [ ]:
year_list = storm_data["Year"].unique()
event_list = storm_data["Event Type"].unique()


clean_tornado_df = pd.DataFrame()
clean_tornado_df = storm_data[storm_data["Event Type"]=="Tornado"]


#clean_storm_data_mn_df
year_count_df = pd.DataFrame()
for i in year_list:
    year_count = clean_tornado_df[clean_tornado_df["Year"]==i]["Event Type"].count()
    year_count_df = year_count_df.append({"Year":i, "Count":year_count}, ignore_index=True)
year_count_df

x_data = (year_count_df["Year"].astype(int))
y_data = year_count_df["Count"].astype(float)

plt.scatter(x_data,y_data, s = 4)
plt.xlabel("Year")
plt.ylabel("Count")
plt.title("Counts of Tornado Events vs Year")

(slope_N, intercept_N, rvalue_N, pvalue_N, stderr_N) = linregress(x_data,y_data)
regress_values_N = x_data * slope_N + intercept_N
line_eq = "y = " + str(round(slope_N,2)) + "x + " + str(round(intercept_N,2))+", r = " +str(round(rvalue_N,2))+", p="+str(round(pvalue_N,2))
min_x = x_data.min()
max_y = y_data.max()
plt.plot(x_data,regress_values_N,"r-")
plt.annotate(line_eq,(min_x+1,max_y-1),fontsize=11,color="red") #writes to coordinates on graph
plt.show()

# Counts of Hail Events in Minnesota from 1950 to 2020

In [ ]:
clean_hail_df = pd.DataFrame()
clean_hail_df = storm_data[storm_data["Event Type"]=="Hail"]


#clean_storm_data_mn_df
year_count_df = pd.DataFrame()
for i in year_list:
    year_count = clean_hail_df[clean_hail_df["Year"]==i]["Event Type"].count()
    year_count_df = year_count_df.append({"Year":i, "Count":year_count}, ignore_index=True)
year_count_df

x_data = (year_count_df["Year"].astype(int))
y_data = year_count_df["Count"].astype(float)

plt.scatter(x_data,y_data, s = 4)
plt.xlabel("Year")
plt.ylabel("Count")
plt.title("Counts of Hail Events vs Year")

(slope_N, intercept_N, rvalue_N, pvalue_N, stderr_N) = linregress(x_data,y_data)
regress_values_N = x_data * slope_N + intercept_N
line_eq = "y = " + str(round(slope_N,2)) + "x + " + str(round(intercept_N,2))+", r = " +str(round(rvalue_N,2))+", p="+str(round(pvalue_N,2))
min_x = x_data.min()
max_y = y_data.max()
plt.plot(x_data,regress_values_N,"r-")
plt.annotate(line_eq,(min_x+1,max_y-1),fontsize=11,color="red") #writes to coordinates on graph
plt.show()

# Counts of Thunderstorm Wind Events in Minnesota from 1950 to 2020

In [ ]:
clean_thunderstorm_wind_df = pd.DataFrame()
clean_thunderstorm_wind_df = storm_data[storm_data["Event Type"]=="Thunderstorm Wind"]


#clean_storm_data_mn_df
year_count_df = pd.DataFrame()
for i in year_list:
    year_count = clean_thunderstorm_wind_df[clean_thunderstorm_wind_df["Year"]==i]["Event Type"].count()
    year_count_df = year_count_df.append({"Year":i, "Count":year_count}, ignore_index=True)
year_count_df

x_data = (year_count_df["Year"].astype(int))
y_data = year_count_df["Count"].astype(float)

plt.scatter(x_data,y_data, s = 4)
plt.xlabel("Year")
plt.ylabel("Count")
plt.title("Counts of Thunderstorm/Wind Events vs Year")

(slope_N, intercept_N, rvalue_N, pvalue_N, stderr_N) = linregress(x_data,y_data)
regress_values_N = x_data * slope_N + intercept_N
line_eq = "y = " + str(round(slope_N,2)) + "x + " + str(round(intercept_N,2))+", r = " +str(round(rvalue_N,2))+", p="+str(round(pvalue_N,2))
min_x = x_data.min()
max_y = y_data.max()
plt.plot(x_data,regress_values_N,"r-")
plt.annotate(line_eq,(min_x+1,max_y-1),fontsize=11,color="red") #writes to coordinates on graph
plt.show()

# Total Count of Tornado, Thunderstorm/Wind, and Hail Events in Minnesota from 1950 to 2020

In [ ]:
# create dataframe with just tornado, thunderstorm wind, and hail events only
main_events = ['Tornado', 'Thunderstorm Wind', 'Hail']
storm_main_events = storm_data.loc[storm_data['Event Type'].isin(main_events)]
storm_main_events

year_count_df = pd.DataFrame()
for i in year_list:
    year_count = storm_main_events[storm_main_events["Year"]==i]["Event Type"].count()
    year_count_df = year_count_df.append({"Year":i, "Count":year_count}, ignore_index=True)
year_count_df

x_data = (year_count_df["Year"].astype(int))
y_data = year_count_df["Count"].astype(float)

plt.scatter(x_data,y_data, s = 4)
plt.xlabel("Year")
plt.ylabel("Count")
plt.title("Counts of Tornado, Thunderstorm Wind, and Hail Events vs Year")

(slope_N, intercept_N, rvalue_N, pvalue_N, stderr_N) = linregress(x_data,y_data)
regress_values_N = x_data * slope_N + intercept_N
line_eq = "y = " + str(round(slope_N,2)) + "x + " + str(round(intercept_N,2))+", r = " +str(round(rvalue_N,2))+", p="+str(round(pvalue_N,2))
min_x = x_data.min()
max_y = y_data.max()
plt.plot(x_data,regress_values_N,"r-")
plt.annotate(line_eq,(min_x+1,max_y-1),fontsize=11,color="red") #writes to coordinates on graph
plt.show()

# Number of Tornado, Thunderstorm Wind, and Hail Events per Year

In [ ]:
storm_main_events_graph = storm_main_events
plt.bar(x_data, y_data)
plt.xlabel("Year")
plt.ylabel("Number of Events")
plt.title("Number of Tornado, Thunderstorm Wind, and Hail Events per Year")
plt.show()


In [ ]:
# weather events from 1950-1995 (when only tornadoes, thunderstorm wind, and hail events were recorded)

storm_events_prior_df = storm_data.loc[((storm_data['Year'] >= 1950) & (storm_data['Year'] < 1995))]
storm_events_prior_df

# keeping only year and event type
storm_events_prior_df = storm_events_prior_df[['Year','Event Type']]
storm_events_prior_df

# count_storms = storm_events_prior_df.groupby('Event Type').count()
# count_storms

In [ ]:
# create pie chart for weather events prior to 1995

In [ ]:
# weather events from 1995-2020
later_storm_events_df = storm_data.loc[((storm_data['Year'] > 1995))]
later_storm_events_df

# keeping only year and event type
later_storm_events_df = later_storm_events_df[['Year', 'Event Type']]
later_storm_events_df

# Percentage of Tornado, Thunderstorm Wind, and Hail Events from 1950 to 1995

## We chose to separate out 1950 to 1995 because NOAA only tracked these 3 weather events until 1996.

In [ ]:
# weather events from 1950-1995 (when only tornadoes, thunderstorm wind, and hail events were recorded)

storm_events_prior_df = storm_data.loc[((storm_data['Year'] >= 1950) & (storm_data['Year'] < 1995))]
storm_events_prior_df

# keeping only year and event type
storm_events_prior_df = storm_events_prior_df[['Year','Event Type']]
storm_events_prior_df

storm_unique_prior = storm_events_prior_df.groupby('Event Type').count()
storm_unique_prior = storm_unique_prior.rename(columns={
    'Event Type': 'Event Type',
    'Year': 'Number of Events'
})
storm_unique_prior['Number of Events'] = pd.to_numeric(storm_unique_prior['Number of Events'])
storm_unique_prior.sort_values(by=['Number of Events'], ascending=False)

In [ ]:
# create pie chart for weather events prior to 1995
colors = ["red", "blue", "green"]

storm_unique_prior
storm_unique_prior.sort_values(by=['Number of Events'], ascending=False).plot.pie(y='Number of Events', colors = colors, autopct="%1.1f%%", shadow=True, startangle=140, figsize=(15,6))
plt.axis("equal")
plt.title("Percentage of Storm Events from 1950 to 1995")
#plt.savefig("output_data/PieTornadoHailThunderstorm19501995.png")
plt.show()

In [ ]:
# weather events from 1995-2020
later_storm_events_df = storm_data.loc[((storm_data['Year'] > 1995))]
later_storm_events_df

# keeping only year and event type
later_storm_events_df = later_storm_events_df[['Year', 'Event Type']]
later_storm_events_df

storm_unique_later = later_storm_events_df.groupby('Event Type').count()
storm_unique_later = storm_unique_later.rename(columns={
    'Event Type': 'Event Type',
    'Year': 'Number of Events'
})
storm_unique_later['Number of Events'] = pd.to_numeric(storm_unique_later['Number of Events'])
storm_unique_later.sort_values(by=['Number of Events'], ascending=False)


# create pie chart for weather events after 1995
explode = [.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

storm_unique_later.sort_values(by=['Number of Events'], ascending=False).plot.pie(y='Number of Events',labeldistance=None,\
                            autopct="%1.1f%%",textprops={'fontsize': 30}, explode=explode,\
                            pctdistance=1.1,shadow=True, figsize=(50,40))
plt.legend(loc=2, prop={'size': 20})

plt.axis("equal")
plt.title("Percentage of Storm Events from 1996 to 2020", fontsize = 40)
plt.show()

In [ ]:
# Emily's code ends here